# 2017 Stack Overflow developer survey results cleansing.

In [1]:
import os
import zipfile
import pandas as pd

ARCHIVE_PATH = 'data/developer_survey_2017.zip'
RESULTS_PATH = 'data/unpacked/survey_results_public.csv'
SCHEMA_PATH = 'data/unpacked/survey_results_schema.csv'
CLEAN_PATH = 'data/unpacked/cleaned_survey_results_public.csv'
FEATURE_COLUMNS = [
    'Professional',
    'ProgramHobby',
    'Country',
    'University',
    'FormalEducation',
    'MajorUndergrad',
    'YearsProgram'
]
LABEL_NAME = 'DeveloperType'

In [2]:
# Unpack Archive if not already unpacked
if os.path.isfile(RESULTS_PATH) is not True:
    zip_ref = zipfile.ZipFile(ARCHIVE_PATH, 'r')
    zip_ref.extractall('data/unpacked')
    zip_ref.close()

In [3]:
# Parse the local CSV file.
if not os.path.isfile(CLEAN_PATH):
    raw_data = pd.read_csv(
        filepath_or_buffer=RESULTS_PATH,
        header=0,
        low_memory=False
    )
    output = FEATURE_COLUMNS
    output.append(LABEL_NAME)
    raw_data.loc[:, output].to_csv(CLEAN_PATH, index=False)
else:
    raw_data = pd.read_csv(
        filepath_or_buffer=CLEAN_PATH,
        header=0,
        low_memory=False
    )

In [4]:
print(raw_data)

       Respondent                                       Professional  \
0               1                                            Student   
1               2                                            Student   
2               3                             Professional developer   
3               4  Professional non-developer who sometimes write...   
4               5                             Professional developer   
5               6                                            Student   
6               7  Professional non-developer who sometimes write...   
7               8                             Professional developer   
8               9                             Professional developer   
9              10                             Professional developer   
10             11  Professional non-developer who sometimes write...   
11             12                             Professional developer   
12             13                Used to be a professional devel